<a href="https://colab.research.google.com/github/HellenBaldo/BuscaDocumentosGenai/blob/main/BuscaDocumentosGenai.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Criando um sistema para busca em documentos usando embeddings e a Gemini API

In [1]:
!pip install -U -q google-generativeai

In [3]:
#Importações e configurações iniciais
import numpy as np
import pandas as pd
import google.generativeai as genai

from google.colab import userdata
api_key = userdata.get('SECRET_KEY')
genai.configure(api_key=api_key)

Abaixo ele lista os modelos disponíveis.

Disponível Guia de embeddings :
https://ai.google.dev/gemini-api/docs/embeddings?hl=pt-br

Gemini API: Embeddings Quickstart  : https://colab.research.google.com/github/google-gemini/cookbook/blob/main/quickstarts/Embeddings.ipynb

In [4]:
for m in genai.list_models():
  if 'embedContent' in m.supported_generation_methods:
    print(m.name)

models/embedding-001
models/text-embedding-004


task_type (Tipo de tarefa) disponível em : https://ai.google.dev/gemini-api/tutorials/document_search?hl=pt-br#api_changes_to_embeddings_with_model_embedding-001

Busca de documentos: RETRIEVAL_DOCUMENT

Busca de consulta: RETRIEVAL_QUERY

Etc.

In [5]:
#Exemplo de embedding
title = "A próxima geração de IA para desenvolvedores e Google Workspace"
sample_text = ("Título: A próxima geração de IA para desenvolvedores e Google Workspace"
    "\n"
    "Artigo completo:\n"
    "\n"
    "Gemini API & Google AI Studio: Uma maneira acessível de explorar e criar protótipos com aplicações de IA generativa")

embeddings = genai.embed_content(model="models/embedding-001",
                                 content=sample_text,
                                 title=title,
                                 task_type="RETRIEVAL_DOCUMENT")

print(embeddings)

{'embedding': [0.051496185, -0.0365346, -0.0186796, 0.019455563, 0.06721682, -0.003868702, -0.027666211, -0.01814636, 0.062080767, 0.062902406, 0.012042811, 0.01314806, -0.045869622, -0.022034725, 0.009636108, -0.024947647, 0.021764254, -0.011328052, -0.03484635, -0.00094051583, 0.009192941, 0.0068539544, -0.029636258, -0.06623153, -0.019914951, 0.020645097, 0.015273816, -0.038156737, -0.032827046, 0.02755795, -0.049995217, 0.05074531, -0.034130614, 0.010284468, -0.036612358, -0.040648427, -0.026028484, -0.044465132, -0.002034308, -0.0036913794, 0.0061324253, -0.084707916, -0.009676676, 0.02611583, -0.0035269416, -0.021471972, 0.04472568, 0.04406418, 0.01429429, -0.058487777, 0.027538216, 0.023262272, 0.066298164, -0.041965708, -0.0041519217, -0.009309915, 0.022716599, -0.032830838, 0.02376307, 0.0006589484, -0.0012957085, 0.023344554, -0.016953444, 0.052708756, 0.01914495, -0.05771819, -0.04336735, -9.953024e-05, 0.011018355, 0.048855364, 0.009574091, 0.012556762, 0.060748216, -0.0642

In [7]:
#Listagem de documentos que serão buscados.
#Insere os documentos na memoria, separado por Título e Conteúdo (que é uma estrutura de chave/valor).
DOCUMENT1 = {
    "Título": "Operação do sistema de controle climático",
    "Conteúdo": "O Googlecar tem um sistema de controle climático que permite ajustar a temperatura e o fluxo de ar no carro. Para operar o sistema de controle climático, use os botões e botões localizados no console central.  Temperatura: O botão de temperatura controla a temperatura dentro do carro. Gire o botão no sentido horário para aumentar a temperatura ou no sentido anti-horário para diminuir a temperatura. Fluxo de ar: O botão de fluxo de ar controla a quantidade de fluxo de ar dentro do carro. Gire o botão no sentido horário para aumentar o fluxo de ar ou no sentido anti-horário para diminuir o fluxo de ar. Velocidade do ventilador: O botão de velocidade do ventilador controla a velocidade do ventilador. Gire o botão no sentido horário para aumentar a velocidade do ventilador ou no sentido anti-horário para diminuir a velocidade do ventilador. Modo: O botão de modo permite que você selecione o modo desejado. Os modos disponíveis são: Auto: O carro ajustará automaticamente a temperatura e o fluxo de ar para manter um nível confortável. Cool (Frio): O carro soprará ar frio para dentro do carro. Heat: O carro soprará ar quente para dentro do carro. Defrost (Descongelamento): O carro soprará ar quente no para-brisa para descongelá-lo."}

DOCUMENT2 = {
    "Título": "Touchscreen",
    "Conteúdo": "O seu Googlecar tem uma grande tela sensível ao toque que fornece acesso a uma variedade de recursos, incluindo navegação, entretenimento e controle climático. Para usar a tela sensível ao toque, basta tocar no ícone desejado.  Por exemplo, você pode tocar no ícone \"Navigation\" (Navegação) para obter direções para o seu destino ou tocar no ícone \"Music\" (Música) para reproduzir suas músicas favoritas."}

DOCUMENT3 = {
    "Título": "Mudança de marchas",
    "Conteúdo": "Seu Googlecar tem uma transmissão automática. Para trocar as marchas, basta mover a alavanca de câmbio para a posição desejada.  Park (Estacionar): Essa posição é usada quando você está estacionado. As rodas são travadas e o carro não pode se mover. Marcha à ré: Essa posição é usada para dar ré. Neutro: Essa posição é usada quando você está parado em um semáforo ou no trânsito. O carro não está em marcha e não se moverá a menos que você pressione o pedal do acelerador. Drive (Dirigir): Essa posição é usada para dirigir para frente. Low: essa posição é usada para dirigir na neve ou em outras condições escorregadias."}

documents = [DOCUMENT1, DOCUMENT2, DOCUMENT3]

In [10]:
# df variavél que faz Conversão do 'documents' para o formato DataFrame usando o Pandas (pd.DataFrame).
df = pd.DataFrame(documents)
# Para não ter problema com assentuação, segue forma de renomear/definir nome das colunas, abrindo um 'lista', e para cada coluna insere um nome:
df.columns = ["Titulo", "Conteudo"]
df

,Titulo,Conteudo
0,Operação do sistema de controle climático,O Googlecar tem um sistema de controle climáti...
1,Touchscreen,O seu Googlecar tem uma grande tela sensível a...
2,Mudança de marchas,Seu Googlecar tem uma transmissão automática. ...


In [11]:
# Variavel que recebe o modelo.
model = "models/embedding-001"

Função automatizar em Python para criar uma nova coluna, que recebe os embedding dos documentos:

* Acessa cada uma das linhas, procurando Titulo e o Texto, para ele olhar aquele conteúdo e gerar o embedding, e armazenar em cada uma das linhas para gerar aquele conteúdo.
* O Pandas possui alguns mecanismos que ele paraleliza e acelera essa processo, pois se fossemos usar um for para ir de linha em linha, o processo iria levar muito tempo em casos de grandes quantidades de documentos.

In [13]:
# Retorna a geração de conteúdo embed (embed_content).
def embed_fn(title, text):
  return genai.embed_content(model=model,
                                 content=text,
                                 title=title,
                                 task_type="RETRIEVAL_DOCUMENT")["embedding"]

Explicação da função abaixo:

* **df["Embeddings"]** : Acessa o DataFrame e cria a nova coluna chamada Embeddings, que será o nome da função.

* **df.apply** : Dentro da função Embeddings,  vamos usar a função 'apply' do Pandas, para criar a estrutura 'embed_fn', para cada linha que tem no DataFrame.

* **(lambda row: embed_fn** : Em seguida, usa-se o 'lambda', para fazer com que essa função olhe para cada linha e aplique o método de Embeddings, que é parecido com um for, para cada linha que há ali dentro, aplica essa função 'lambda row: e chama a função, dentro da função embed_fn

*  **'(row["Titulo"], row["Conteudo"])'** : abre o paranteses para passar o argumento que irá acessar cada linha e de cada coluna. Então, pega a primeira linha da coluna identificada como 0, dessa linha quebro em duas variaveis que é a coluna 'Titulo' e a coluna 'Conteudo'.

* Então, na função temos **(title, text)** que recebem os argumentos da nossa função anterior que é o **Titulo** e o **Conteudo**, seguindo essa sequencia para a função rodar conforme o esperado.

* **axis=1)** : refere-se ao 'eixo', para poder acessar as colunas e aplicar para cada uma das linhas.

Basicamente isso funciona da seguinte forma:

- Cada linha representa um documento, iniciando na linha 0, pega o Titulo e pega o Conteúdo, e roda a função 'embed_fn'
- Ela irá retornar alguma coisa, e o que ela retornar você insere nessa mesma row/linha 0, em uma nova coluna chamada 'Embeddings'.
- Daí na próxima linha, faça a mesma coisa.

Sendo assim, ao usar essa estrutura, é possível evitar erros ao criar novas colunas com o DataFrame Pandas. Usando o método 'apply' do DataFrame, e usar uma função 'lambda' para garantir que iremos rodar a função linha por linha. Porque o que normalmente acontece ao montar essa estrutura diferente ou errada, você pode acabar tendo o *mesmo Embedding em todas as linhas*.


In [16]:
df["Embeddings"] = df.apply(lambda row: embed_fn(row["Titulo"], row["Conteudo"]), axis=1)
df

,Titulo,Conteudo,Embeddings
0,Operação do sistema de controle climático,O Googlecar tem um sistema de controle climáti...,"[-0.011010795, -0.026731547, -0.036728486, 0.0..."
1,Touchscreen,O seu Googlecar tem uma grande tela sensível a...,"[0.015715627, -0.040427547, 0.011117627, 0.002..."
2,Mudança de marchas,Seu Googlecar tem uma transmissão automática. ...,"[-0.009390755, -0.022475218, -0.0024638234, 0...."


O proximo passo é fazer consultas com um conhecimento semantico, ou seja um conhecimento de significado.

Sendo assim, vamos construir nossa estrutura para buscar nossa resposta, criando uma função que irá fazer isso.

Até o momento temos os nossos documentos, representados na forma de embedding, para conseguir fazer uma pergunta em Portugues em texto puro, ou seja, procurar uma informação dentro desses documentos, temos que representar a 'minha pergunta' no mesmo formato que eu identifiquei o texto. Ou seja, tenho que gerar um embedding das minhas perguntas também.

* A função **gerar_e_buscar_consulta** que possui parâmetros de consulta, o DataFrame e a variavel do modelo **(consulta, base, model)**.
* A consulta que vamos receber irá passar pelo embedding.
* Porém ao invés que retornar como antes, iremos salvar o resultado em uma variavel chamada **'embedding_da_consulta'**.
* Como estamos fazendo uma consulta para fazer buscas no documento, o tipo de tarefa 'task_type' muda para **RETRIEVAL_QUERY**.

Como já estruturamos o embedding da consulta, agora para encontrar o contexto, faremos o calculo da distancia, entra cada documento o embedding que estamos treinando, para poder entender a questão de semântica (produtos escalares).

É essa é uma estrutura de comparação, que poderá ser utilizada para qualquer cenário de buscas de informação usando embedding.
Basicamente, iremos pegar todos os documentos embedding e fazer um 'empilhamento' de embeddings, que iremos calcular com o produtos escalares, e realizar a comparação, que seria o calculo da distância.

Exemplo: temos 3 documentos, e iremos mandar 1 pergunta que é a nossa consulta, nisso iremos fazer o calculo de distancia "para para", a pergunta para o documentos 1, a pergunta para o documento 2, e a pergunta contra o documento 3, etc. Nisso será analisado qual é o documento 'mais próximo' para a busca retornar o melhor contexto.

Para esse produto escalar, iremos usar uma biblioteca especifica, chamado NUMPY, para trabalhar com vetores e funções matematicas. O **np.dot** fazer esse produto escalar.

A varíavel **produtos_escalares** irá empilhar os embeddings com o **np.stack** (que é a pilha), e iremos colocar os embeddings do DataFrame e os embeddings que pegamos da consulta, que é a variável 'embedding_da_consulta'.

* Pega a coluna anterior que criamos **(df["Embeddings"])** dentro do stack que irá empilhar. E fora do stack chamamos a consulta **embedding_da_consulta**.

* Basicamente, o np.dot irá fazer a multiplicação do **(df["Embeddings"])**, em cada um dos documentos (no exemplo temos 3, mas poderia ser 10mil).


O proximo ponto é identificar qual será o argumento máximo desses Embeddings.
Dentro da variavel **produtos_escalares**, você terá uma referencia de cada um dos textos que temos dentro do nosso DataFrame, e um valor que representa a distância.

Quando usamos a próxima função que é o **np.argmax** do NUMPY, nos vamos buscar dentro dessa lista **produtos_escalares**, qual que se destaca mais ali dentro, e pegar somente o ID dele. E vamos salvar isso, em uma variavel chamada **indice**, pois ele irá trazer o indice do Token, em que ele identificou a maior similaridade de contexto.

Como é uma função precisamos retornar alguma coisa. Então o return irá acessar o indice daquele DataFrame, e irá trazer o texto identificado.

 * **df.iloc[indice]["Conteudo"]** : busca o indice que acabamos que criar, e para buscar esse indice qual que é a coluna de referencia de texto que está o nosso Conteudo.

 Resumindo: pega a consulta e gera um Embedding dela, depois faz o calculo de distancia para ver qual está mais próxima e qual documente tem mais haver, e pega o indice do Token mais próximo. Depois vai até o DataFrame (df), e no indice retorna o texto que está na coluna Conteúdo.

 Essa mecanica, é usando a mesma estrutura de tabela/DataFrame, usando a coluna de Embeddings como referencia matematica para o modelo entender, porém na hora de queremos mostrar a resposta para o usuário, não faz sentido mandarmos uma sequencia de número. Então buscamos o conteúdo de texto em português, para termos a resposta para a pergunta que foi feita.








In [17]:

def gerar_e_buscar_consulta(consulta, base, model):
  embedding_da_consulta = genai.embed_content(model=model,
                                 content=consulta,
                                 task_type="RETRIEVAL_QUERY")["embedding"]

  produtos_escalares = np.dot(np.stack(df["Embeddings"]), embedding_da_consulta)

# Busca o argumento máximo usando o argmax.
  indice = np.argmax(produtos_escalares)
  return df.iloc[indice]["Conteudo"]

**Nessa etapa, vamos fazer uma consulta.**

Precisamos chamar a função **gerar_e_buscar_consulta**.

Para isso, vamos usar três argumentos. Precisamos de uma variavel que representa a consulta, que chamaremos de 'trecho', a variavel de base que é o nosso DataFrame df, e o model, sendo que a variavel df e model já estão na memória.

E uma varíavel com a pergunta que iremos fazer, que chamaremos de 'consulta'.



In [18]:
consulta = "Como faço para trocar marchas em um carro do Google?"

trecho = gerar_e_buscar_consulta(consulta, df, model)
print(trecho)

Seu Googlecar tem uma transmissão automática. Para trocar as marchas, basta mover a alavanca de câmbio para a posição desejada.  Park (Estacionar): Essa posição é usada quando você está estacionado. As rodas são travadas e o carro não pode se mover. Marcha à ré: Essa posição é usada para dar ré. Neutro: Essa posição é usada quando você está parado em um semáforo ou no trânsito. O carro não está em marcha e não se moverá a menos que você pressione o pedal do acelerador. Drive (Dirigir): Essa posição é usada para dirigir para frente. Low: essa posição é usada para dirigir na neve ou em outras condições escorregadias.


Os passos acima, auxilia a se proteger 100% do risco de alucinação de modelo. Pois não estamos pedindo o modelo 'gerar uma resposta', pois usamos o modelo para criar representações de **Embeddings**, mas a minha resposta vem de um documento que eu criei e que tenho controle da explicação, e assim tenho a certeza de que o que está escrito ali não está tendo nenhum risco de um desvio de informação, ou algo inventado.

Então isso é uma das abordagens que você pode usar a IA Generativa, o Gemini, para acelerar seu processo, mas que você tem controle do dado da resposta que vai ser devolvida.

Em uma empresa, que possui um cenário que tem muitas informações, como por exemplo, informações de RH de (férias, gerar holerite, segunda via de algo, etc) e geral, Guias, ajuda para resolução de problemas, etc. Qualquer informação que é processual interna da empresa, e que é muito importante que não tenha um desvio de informação, pois ao vincular isso a um documento, garante-se que terá a resposta correta, para resposta da pergunta. É seguro, pois você sabe o que está sendo alimentado no modelo, se preparar muito bem, os dados que estarão sendo inseridos no modelo, ele terá a qualidade de acordo com os dados que está entrando dentro. Nisso é importante ter um cuidado ao criar essa parte de **Embeddings**, pois a qualidade do Conteúdo que irá ser gerado, é baseado em cima disso.

Vale ressaltar o **TRADE-OFF**, ao ter segurança na informação, mais tendo resposta menos criativa. Todas as vezes que for executado a célula da consulta, sempre a resposta será exatamente o mesmo texto, porque ele busco no mesmo documento.

Se o cenário que você está, tem uma tolerancia maior para uma resposta não tão exata, mas a criatividade é muito importante, então é algo que deve ser pesado a melhor forma de implementar (existe também a questão do RAG).



Nessa próxima etapa, vamos fazer uma junção desses modelos mais seguros, junto com a parte "mais criativa". Podemos usar o próprio Gemini, criando uma nova interação com o Gemini, usando um Prompt, e usar a variavel 'trecho' que é a resposta dos documentos, pedindo para ele reescrever essa resposta de forma diferente.

Temos um prompt iniciando que recebe f"String", texto e uma variavel juntos, pedindo para reescrever o 'trecho'.

In [22]:
# Configurações do modelo
generation_config = {
  "temperature": 0,
  "candidate_count": 1
}

In [23]:
prompt = f"Reescreva esse texto de uma forma mais descontraída, sem adicionar informações que não façam parte do texto: {trecho}"

model_2 = genai.GenerativeModel("gemini-1.0-pro",
                                generation_config=generation_config)
response = model_2.generate_content(prompt)
print(response.text)

**Guia do Googlecar para Descolados**

Seu Googlecar é tipo um carro automático maneiro. Pra trocar as marchas, é só dar um toque na alavanca e escolher o que você quer:

* **Estacionar:** Quando você tá parado e quer que o carro fique quietinho. As rodas ficam travadas e ele não mexe nem um centímetro.
* **Ré:** Pra dar aquela ré maneirinha.
* **Neutro:** Quando você tá parado no farol ou no trânsito. O carro tá desligado e não vai se mexer até você pisar no acelerador.
* **Dirigir:** Pra acelerar pra frente.
* **Baixa:** Pra quando você tá dirigindo na neve ou em lugares escorregadios.


Aqui misturamos as duas coisas, pegamos a busca de documentos que construimos usando **Embeddings**, gerou um resultado confiavel e seguro, e depois usamos o Gemini para reescrever de forma mais fluída.

Aqui pode ser definido a forma que o Gemini irá reescrever, se quer em JSON, se quer em tópicos, em poemas, em outros idiamos, etc.

É necessário tomar cuidado com a LGPD, informações que são confidenciais e pessoais, precisa ter um contexto de cuidado. Pois as informações que trocam com o Prompt, pode ser acessada pelo Google, então se for algo sigiloso do projeto que esteja trabalhando, se tem propriedade intelectual envolvida, não é o melhor tipo de informação para fazer essas experimentações, pois é melhor serem usadas em um cenário mais formal e controlado (exemplo: com a Google Cloud, onde somente você terá acesso).